## Probabilistic Programming 4: Assignment

In this assignment, you will be simultaneously estimating states and process noise for a linear Gaussian dynamical system.

In [ ]:
using Pkg
Pkg.activate("workspace/")
Pkg.instantiate();

In [ ]:
using JLD
using ForneyLab
using Plots

In [ ]:
# Load previously generated signal
signal = load("util/signal.jld");

# Extract variables
T = signal["T"]
Y = signal["Y"]
γ = signal["γ"]

# Visualize data
plot(1:T, Y, color="black", label="observations", markersize=6, xlabel="time (t)", ylabel="signal", size=(800,300))

The data above is from a random walk, i.e. a process defined by the following equations:

$$\begin{align*}
x_k =&\ x_{k-1} + w_k \\
y_k =&\ x_k + v_k \, .
\end{align*}$$

The variables $w$ and $v$ are process noise and measurement noise respectively, with $w \sim \mathcal{N}(0, \tau^{-1})$ and $v \sim \mathcal{N}(0, \gamma^{-1})$. The parameter $\tau$ represents process noise _precision_ and the parameter $\gamma$ represents measurement noise _precision_. We do know the value of $\gamma$, but not of $\tau$. With Bayesian inference, we can simultaneously estimate the time-varying states and the time-invariant process noise precision. You need only to cast the equations to a generative model and pose a prior for the unknown variables.

### **1) Finish the model specification below.**

Add a variable `τ` with the prior for the process noise _precision_ and a variable `x_k` with the state transition distribution. Note that the prior for `τ` should contain two placeholders: `placeholder(:1)` and `placeholder(:τ2)`.

Remember to use `@RV var ~ ...` to define a variable distributed according to a desired distributon. Options include `GaussianMeanVariance`, `GaussianMeanPrecision`, `Gamma`, `Wishart`, `Beta`, `Bernoulli`, `Categorical` and `Dirichlet`.

In [ ]:
# Initialize a graph
graph = FactorGraph()

# Define initial state prior
@RV x_kmin1 ~ GaussianMeanPrecision(placeholder(:m_x_kmin1), 
                                    placeholder(:w_x_kmin1))

### YOUR CODE HERE
    
# Observation likelihood
@RV y_k ~ GaussianMeanPrecision(x_k, γ)
    
# Tell FL that y is observed
placeholder(y_k, :y_k);

# Visualize subgraph
ForneyLab.draw(graph)

In [ ]:
# This cell compiles an inference algorithm based on your model specification (3 points)
q = PosteriorFactorization(x_k, x_kmin1, τ, ids=[:x :x_kmin1 :τ])
algorithm = messagePassingAlgorithm()
source_code = algorithmSourceCode(algorithm)
eval(Meta.parse(source_code));

In [ ]:
# SPOILER ALERT! 
# This script contains clues on specifying the model correctly.
# Do not look into it if you don't want the assignment spoiled!
include("util/run_inference.jl")

Your generated plot should look something like this:

![](figures/filtered_signal.png)